In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.8/328.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.8 MB/s eta 0:00:00


In [4]:
import json
import os
import tiktoken
from google.colab import userdata
from openai import OpenAI

In [5]:
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
BASE_PATH = "/content/drive/MyDrive/Colab/NER"

## Calculate price using tiktoken

### Get total paragraphs

In [9]:
def read_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return file.read()

In [10]:
filepath = f"{BASE_PATH}/wikipedia_summary.txt"
text = read_file(filepath)
paragraphs = text.replace("\n\n", "\n").split("\n")
print(f"The file contains {len(paragraphs)} paragraphs.\n")

The file contains 110038 paragraphs.



### Calculate price

[Pricing OpenAI](https://openai.com/api/pricing/)

In [ ]:
# Pricing in USD
model_pricing_by_1K_tokens = {
  "gpt-4o-mini": {
    "input_tokens": {
        "api": 0.00015,
        "bacth_api": 0.00008
    },
    "output_tokens": {
        "api": 0.0006,
        "bacth_api": 0.0003
    }
  },
  "gpt-4o": {
    "input_tokens": {
        "api": 0.005,
        "bacth_api": 0.0025
    },
    "output_tokens": {
        "api": 0.015,
        "bacth_api": 0.0075
    }
  },
}


In [23]:
def num_tokens_per_text(text, model):
  try:
      encoding = tiktoken.encoding_for_model(model)
  except KeyError:
      print("Warning: model not found. Using cl100k_base encoding.")
      encoding = tiktoken.get_encoding("cl100k_base")

  token_integers = encoding.encode(text)
  print(f"Model: {model}\nEncoding: {encoding.name}")
  return len(token_integers)

In [ ]:
def estimate_cost(num_tokens, fee_per_token, tokens=1000):
  final_price = (num_tokens * fee_per_token) / tokens
  return final_price

In [ ]:
model = "gpt-4o-mini"
model_pricing = model_pricing_by_1K_tokens[model]
batch_format_sample = '{"custom_id": "wk-499", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o-mini", "messages": [{"role": "system", "content": "Eres un experto generando y creando etiquetas para un texto en español."}, {"role": "user", "content": "Como especialista anotador de información, tu tarea es detectar entidades de todo tipo en un texto para generar un dataset de entrenamiento de un modelo de IA.\nEtiqueta todo tipo de entidades, conceptos e ideas como te sea posible, saliéndote del esquema general de los datasets NER.Sientete libre de crear nuevos tipos de entidades que pueden no existir en tareas NER tradicionales.\nEl formato de salida es el siguiente: <palabra> <> <nombre de la entidad> <> <descripcion de la entidad>\nEjemplo: España <> País <> Nación localizada en el sur de EuropaEl texto es el siguiente: Elvira Malagarriga Armart, también conocida como Elvira Ormart, fue una pintora española especializada en retratos, paisajes y flores. Nacida en Barcelona en 1886, estudió en la Escuela de Bellas Artes de La Llotja y luego se formó en París, donde asistió a la Académie Julian. Malagarriga expuso su obra en diversos lugares de España y Francia, incluyendo el Salón de París, la Asociación de Artistas Italianos en Florencia, y en Sala Parés en Barcelona. Su obra incluye retratos, como el de su padre y su hermana, así como paisajes y flores. Su estilo fue elogiado por la revista Feminal, que comparó su habilidad con la de otras pintoras femeninas de la época. El Museo Nacional de Arte de Cataluña conserva una de sus obras, un paseo con una casa señorial."}], "max_tokens": 2500}}'

if model_pricing:
  num_tokens = num_tokens_per_text(batch_format_sample, model)
  print(f"The text contains: {num_tokens} tokens.")

  # Estimate cost
  input_price_with_api = estimate_cost(num_tokens, model_pricing["input_tokens"]["api"])
  print(f"Estimated cost with API per paragraph: {input_price_with_api:2f}$.")

  input_price_with_api_per_100k_texts = input_price_with_api * len(paragraphs)
  print(f"Estimated cost with API per {len(paragraphs)} paragraphs: {input_price_with_api_per_100k_texts:2f}$.")

  input_price_with_batch_api = estimate_cost(num_tokens, model_pricing["input_tokens"]["bacth_api"])
  print(f"Estimated cost with Batch API per paragraph: {input_price_with_batch_api:2f}$.")

  input_price_with_batch_api_per_100k_texts = input_price_with_batch_api * len(paragraphs)
  print(f"Estimated cost with Batch API per {len(paragraphs)} paragraphs: {input_price_with_batch_api_per_100k_texts:2f}$.")
else:
  print(f"Model {model} not found.")

Model: gpt-4o-mini
Encoding: o200k_base
The text contains: 396 tokens.
Estimated cost with API per paragraph: 0.000059$.
Estimated cost with API per 110038 paragraphs: 6.536257$.
Estimated cost with Batch API per paragraph: 0.000032$.
Estimated cost with Batch API per 110038 paragraphs: 3.486004$.




---





## Generate NER tags

### Create promt for generate NER tags

***Run a test for the prompt***

In [19]:
sample_text = paragraphs[45]

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
        "role": "system",
        "content": "Eres un experto generando y creando etiquetas para un texto en español.",
    },
    {
        "role": "user",
        "content": (
            "Como especialista anotador de información, tu tarea es detectar entidades de todo tipo en un texto para generar un dataset de entrenamiento de un modelo de IA.\n"
            "Etiqueta todo tipo de entidades, conceptos e ideas como te sea posible, saliéndote del esquema general de los datasets NER."
            "Sientete libre de crear nuevos tipos de entidades que pueden no existir en tareas NER tradicionales.\n"
            "Muy importante: NO etiquetes palabras comunes como artículos ('El', 'La', 'Los', 'Las'). En su lugar, etiqueta la palabra o frase a la que hace referencia el artículo.\n"
            "El formato de salida es el siguiente: <palabra> <> <nombre de la entidad> <> <descripcion de la entidad>"
            "Ejemplo: España <> País <> Nación localizada en el sur de Europa."
            f"El texto es el siguiente: {sample_text}"
        ),
    },
  ],
  temperature=0.7,
  max_tokens=1024,
  top_p=0.7,
  frequency_penalty=0,
  presence_penalty=0
)

response.choices[0].message.content.replace("\n\n", "\n").split("\n")

['álbum <> Obra musical <> Colección de canciones grabadas por un artista o banda.  ',
 'En Directo <> Título de álbum <> Nombre del álbum grabado por la banda Cicatriz.  ',
 'banda <> Grupo musical <> Conjunto de músicos que tocan juntos.  ',
 'Cicatriz <> Banda de rock <> Grupo musical de rock originario de España.  ',
 '1994 <> Año <> Año en el que fue grabado el álbum.  ',
 '1995 <> Año <> Año en el que fue lanzado el álbum.  ',
 'homenaje <> Concepto <> Acto de recordar y honrar a alguien.  ',
 'miembros <> Personas <> Integrantes de la banda Cicatriz.  ',
 'fallecidos <> Estado <> Personas que han muerto.  ',
 'energía <> Concepto <> Vitalidad y fuerza que transmite la música.  ',
 'sonido metalero <> Género musical <> Estilo musical caracterizado por guitarras eléctricas distorsionadas y ritmos intensos.  ',
 'guitarrista <> Músico <> Persona que toca la guitarra.  ',
 'Goar Iñurrieta <> Guitarrista <> Nombre del guitarrista de la banda Cicatriz.  ',
 'estilo heavy <> Estilo mus

## Using Batch API

### Prepare Batch file
- Create a Jsonl file.
- Each request must include a unique custom_id value, which you can use to reference results after completion

Example:


```
{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
{"custom_id": "request-2", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are an unhelpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
```



In [20]:
# Declare global variables
method = "POST"
url = "/v1/chat/completions"
model = "gpt-4o-mini"
max_tokens = 2500
system_prompt = "Eres un experto generando y creando etiquetas para un texto en español."
chunk_size = 500
limit_token = 200000
batch_group = {}

In [21]:
def save_text_in_jsonl(texts, file_path):
  with open(file_path, "w", encoding='utf-8') as json_file:
    for text in texts:
        json_line = json.dumps(text, ensure_ascii=False)
        json_file.write(json_line + '\n')

In [31]:
for i in range(0, len(paragraphs), chunk_size):
  batch_format = []
  chunk = paragraphs[i: i + chunk_size]
  batch_name = f"batch_{i}"
  batch_group[batch_name] = []

  for index, paragraph in enumerate(chunk):
    # To exclude phrases that are not article summaries
    if len(paragraph) > 70:
      batch_group[batch_name].append(paragraph)
      custom_id = f"wk-{index}"
      message = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
          "role": "user",
          "content": (
            "Como especialista anotador de información, tu tarea es detectar entidades de todo tipo en un texto para generar un dataset de entrenamiento de un modelo de IA.\n"
            "Etiqueta todo tipo de entidades, conceptos e ideas como te sea posible, saliéndote del esquema general de los datasets NER."
            "Sientete libre de crear nuevos tipos de entidades que pueden no existir en tareas NER tradicionales.\n"
            "Muy importante: NO etiquetes palabras comunes como artículos ('El', 'La', 'Los', 'Las'). En su lugar, etiqueta la palabra o frase a la que hace referencia el artículo.\n"
            "El formato de salida es el siguiente: <palabra> <> <nombre de la entidad> <> <descripcion de la entidad>"
            "Ejemplo: España <> País <> Nación localizada en el sur de Europa."
            f"\nEl texto es el siguiente: {paragraph}"
          )
        }
      ]
      batch_format.append({
        "custom_id": custom_id,
        "method": method,
        "url": url,
        "body": {
          "model": model,
          "messages": message,
          "max_tokens": max_tokens
        }
      })

  # total_tokens = num_tokens_per_text(str(batch_format), model)
  # if total_tokens > limit_token:
  #   print(f"💥The token amount is greater than that allowed by {model}. Total tokens: {total_tokens}")
  #   break

  filename = f"batch_{i}.jsonl"
  filepath = f"{BASE_PATH}/{filename}"

  save_text_in_jsonl(batch_format, filepath)
  print(f"Saved file: {filename}")


Model: gpt-4o-mini
Encoding: o200k_base
Saved file: batch_0.jsonl
Model: gpt-4o-mini
Encoding: o200k_base
Saved file: batch_500.jsonl
Model: gpt-4o-mini
Encoding: o200k_base
Saved file: batch_1000.jsonl
Model: gpt-4o-mini
Encoding: o200k_base
Saved file: batch_1500.jsonl
Model: gpt-4o-mini
Encoding: o200k_base
Saved file: batch_2000.jsonl
Model: gpt-4o-mini
Encoding: o200k_base
Saved file: batch_2500.jsonl
Model: gpt-4o-mini
Encoding: o200k_base
Saved file: batch_3000.jsonl
Model: gpt-4o-mini
Encoding: o200k_base
Saved file: batch_3500.jsonl
Model: gpt-4o-mini
Encoding: o200k_base
Saved file: batch_4000.jsonl
Model: gpt-4o-mini
Encoding: o200k_base
Saved file: batch_4500.jsonl
Model: gpt-4o-mini
Encoding: o200k_base
Saved file: batch_5000.jsonl
Model: gpt-4o-mini
Encoding: o200k_base
Saved file: batch_5500.jsonl
Model: gpt-4o-mini
Encoding: o200k_base
Saved file: batch_6000.jsonl
Model: gpt-4o-mini
Encoding: o200k_base
Saved file: batch_6500.jsonl
Model: gpt-4o-mini
Encoding: o200k_bas

### Upload Batch file

In [32]:
jsonl_files = [
  f"{BASE_PATH}/{file}"
  for file in os.listdir(BASE_PATH)
  if file.endswith(".jsonl")
]
len(jsonl_files)

222

### Do the following steps for the number of jsonl files

In [34]:
batch_name = "batch_0"
jsonl_file = f"{batch_name}.jsonl"

In [35]:
batch_file = client.files.create(
  file=open(f"{BASE_PATH}/{jsonl_file}", "rb"),
  purpose="batch"
)

batch_file_id = batch_file.id
print(f"Uploaded: {jsonl_file} \nFile id: {batch_file_id}")

Uploaded: batch_0.jsonl 
File id: file-w6JIa9w7fF4B4YIqgZzw3GGH


### Create the batch job
💡One job must be created at a time

In [36]:
batch_response = client.batches.create(
  input_file_id=batch_file_id,
  endpoint="/v1/chat/completions",
  completion_window="24h",
)

batch_job_id = batch_response.id
print(batch_response)

Batch(id='batch_jrzxp7dWTniijDIAgfMuNJ8B', completion_window='24h', created_at=1721639535, endpoint='/v1/chat/completions', input_file_id='file-w6JIa9w7fF4B4YIqgZzw3GGH', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1721725935, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [40]:
output_file_id = client.batches.retrieve(batch_job_id).output_file_id
status = client.batches.retrieve(batch_job_id).status
print(f"Job status: {status}")
print(f"Output file id: {output_file_id}")

Job status: completed
Output file id: file-9foa6xlpeg8BjutdzEx5k7Ze


### Retrieveing the results

In [41]:
output_file = client.files.content(output_file_id)
tagged_texts = []

# Iterate over the response content line by line
for line in output_file.iter_lines():
    try:
        response = json.loads(line)
        data = response['response']['body']['choices'][0]['message']['content']
        tagged_texts.append(data)
    except json.JSONDecodeError:
        print("Skipping non-JSON line:", line)

In [47]:
print(batch_group[batch_name][51])
print("\n")
print(tagged_texts[51])

El artículo describe la isla de Sinjido, ubicada en la provincia de Jeollanam-do, Corea del Sur. Con una extensión de 30,99 km cuadrados, Sinjido cuenta con varios cerros como Sang-san, Nohak-bong y Beom-san. La isla se conecta con el continente a través de un puente construido en 2004 y tiene una longitud de 13 km.


isla de Sinjido <> Isla <> Porción de tierra rodeada de agua.  
Jeollanam-do <> Provincia <> Divisão administrativa de Corea del Sur.  
Corea del Sur <> País <> Nación ubicada en el este de Asia.  
30,99 km cuadrados <> Medida de superficie <> Unidad de medida que indica área.  
Sang-san <> Cerro <> Elevación natural del terreno.  
Nohak-bong <> Cerro <> Elevación natural del terreno.  
Beom-san <> Cerro <> Elevación natural del terreno.  
puente <> Infraestructura <> Estructura que permite la comunicación entre dos puntos separados por un obstáculo.  
2004 <> Año <> Periodo de tiempo que indica el año 2004.  
13 km <> Longitud <> Medida que indica la extensión en línea r

In [48]:
paragraphs = batch_group[batch_name]
if len(tagged_texts) == len(paragraphs):
  dataset = []
  for tagged_text, paragraph in zip(tagged_texts, paragraphs):
    # Remove spaces at the end of each line and filter empty lines
    lines = tagged_text.split('\n')
    ner_tags = [line.strip() for line in lines if line.strip()]

    dataset.append({
        "input": paragraph,
        "output": ner_tags
    })
else:
  print("The number of tagged texts and originals paragraphs are different.")

In [49]:
def save_text_by_concatenating_content(texts, filepath):
  with open(filepath, "a", encoding='utf-8') as json_file:
    for text in texts:
        json_line = json.dumps(text, ensure_ascii=False)
        json_file.write(json_line + '\n')
  print(f"Saved {len(texts)} texts")

In [50]:
save_text_by_concatenating_content(dataset, f"{BASE_PATH}/ner_dataset.jsonl")

Saved 469 texts
